In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
import numpy as np

## Masked Self Attention
#### Use a mask with the output of Q.K' before passing to softmax

### Create Q,K,V Matrix

In [50]:
seq_len = 4
d_model = 6

x = np.random.rand(seq_len,d_model)

d_k = d_model

W_Q = np.random.rand(d_model, d_k)
W_K = np.random.rand(d_model, d_k)
W_V = np.random.rand(d_model, d_k)

Q = x @ W_Q
K = x @ W_K
V = x @ W_V

print("Shape of Q,K,V : ", Q.shape)

Shape of Q,K,V :  (4, 6)


### Calculate Q.K' and apply Mask  
Mask for current dimensions :   
[0 -inf -inf -inf]  
[0   0  -inf -inf]  
[0   0    0  -inf]  
[0   0    0    0 ]

In [51]:
prod = Q @ K.T
print(prod)

[[ 9.01425448 11.97882205 14.57610895 15.83571952]
 [10.41229935 13.87032019 16.92282022 18.21022025]
 [11.56935418 15.44540448 18.87519565 20.18786093]
 [12.59571787 16.72310804 20.40889181 21.86072848]]


In [52]:
#create mask
mask = np.zeros(prod.shape)
print(mask)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [53]:
len_token = prod.shape[0]

for i in range(len_token):
    for j in range(len_token):
        if j>i:
            mask[i,j] = -np.inf

print("Mask : \n",mask)

Mask : 
 [[  0. -inf -inf -inf]
 [  0.   0. -inf -inf]
 [  0.   0.   0. -inf]
 [  0.   0.   0.   0.]]


In [54]:
score = prod + mask
print(score)

[[ 9.01425448        -inf        -inf        -inf]
 [10.41229935 13.87032019        -inf        -inf]
 [11.56935418 15.44540448 18.87519565        -inf]
 [12.59571787 16.72310804 20.40889181 21.86072848]]


### Apply Softmax

In [59]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x/np.sum(exp_x, axis=1, keepdims=True)

In [66]:
attention_weights = softmax(score)
print(attention_weights)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.05305589e-02 9.69469441e-01 0.00000000e+00 0.00000000e+00]
 [6.50108112e-04 3.13568799e-02 9.67993012e-01 0.00000000e+00]
 [7.63480841e-05 4.73479214e-03 1.88806295e-01 8.06382564e-01]]


### Masked Self Attention

In [68]:
print(V)

[[0.94004096 1.89035184 1.66300243 1.05727422 1.14389165 1.56307559]
 [1.24596652 2.19213308 2.02045074 1.24365879 1.65816325 1.9094792 ]
 [1.41630512 2.30365315 2.16990316 1.36422157 1.96047487 2.35212402]
 [1.55718408 2.36651757 2.66231286 1.17371806 1.93455072 2.88388564]]


In [72]:
attention_output = attention_weights @ V
print(attention_output)

[[0.94004096 1.89035184 1.66300243 1.05727422 1.14389165 1.56307559]
 [1.23662645 2.18291953 2.00953765 1.23796836 1.64246225 1.8989033 ]
 [1.41065421 2.29988754 2.16488726 1.36024155 1.95046445 2.33773109]
 [1.52906457 2.35378635 2.56622743 1.21000859 1.93807636 2.77877124]]
